# Quy định khi triển khai GA

- Lựa chọn encoding, mutation(đột biến): mình chọn cách chuyển các layer (weights, bias của các matrix giữa các lớp) thành 1 sequence và coi sequence này là chromosome (bạn có thể hiểu là DNA). Về phần mutation, với mỗi số trong sequence, mình sẽ có 1 xác suất nhỏ số đấy sẽ được cộng với 1 số random nào đó (khá nhỏ so với số này).

- Lựa chọn hàm fitness: thay vì lấy điểm số màn chơi làm hàm fitness, mình đánh giá bot nào tốt hơn bằng cách xem bot nào xử lý các cây xương rồng tốt hơn, nghĩa là lấy số cây xương rồng mà bot vượt qua được làm hàm fitness.

- Lựa chọn population, survival: mình chọn population là 12. Thực ra mình thấy 12 hơi nhỏ và có lẽ 50 là con số tốt hơn, nhưng bởi vì training trên môi trường bình thường nên khá chậm. Ví dụ như 1 cá thể tốt có thể đạt tới 500 điểm, như vậy sẽ mất vài phút để chạy xong, và như vậy 1 generation sẽ mất vài tiếng để pass tới generation mới. Về survival, mình chọn 4 cá thể có hàm fitness cao nhất để nhân giống cho thế hệ sau. Sau khi thực hiện xong project, mình nghĩ nên chọn nhiều cá thể hơn làm cha mẹ, như vậy sẽ tăng thời gian training nhưng có thể đạt được hội tụ tốt hơn. Tuy nhiên kết quả project dù sao cũng tốt với thiệt lập như này.

- Lựa chọn cách crossover: mình chọn cách cắt đôi sequence để thực hiện crossover. Do ghép từ 2 cặp cắt đôi (là có cắt weights và bias) nên sẽ có 4 kết quả. Mình chọn random trong 4 kết quả này. Bạn có thể hình dung thông qua minh hoạ dưới đây. Do chỉ có 4 cha mẹ cho 1 generation mới, mình lựa chọn 1 cá thể có thể nhân giống với chính nó.

# Code phần GA

In [2]:
import numpy as np

In [3]:
N_X = 3
N_H = 3
N_Y = 1

POP_SIZE = 12
MUTATION_PROB = 0.1
N_SIZE = 4
RANDOM_SET = [6, 5, 4, 3]
BODY_KEYS = ["W1", "W2", "b1", "b2"]
MUTATION_RANGE = [0.005, 0.5, 0.2, 0.05]


## Converter 

##### Cần 1 hàm chuyển các parameters_set (tức là các hệ số trong NN) sang sequence (chromosome/DNA) và 1 hàm theo chiều ngược lại.

![Alt text](image-5.png)

In [ ]:
def cv_to_sequence(body):
    """
    cv body aka parameters_set to ADN aka sequence
    """
    sequence = []
    for key in BODY_KEYS:
        sequence.append(body[key])
    sequence_str = str(sequence)
    sequence_str = sequence_str.replace("[", "").replace("array", "")
    sequence_str = sequence_str.replace("]", "").replace("\n", "")
    sequence_str = sequence_str.replace("(", "").replace(")", "")
    sequence_str = sequence_str.replace(" ", "")
    sequence_adj = map(float, sequence_str.split(","))
    return sequence_adj


`body= {'W1': array([[-2.25577063e-03,  4.06706987e-03,  1.06554360e-02],
       [ 1.33614910e-02, -8.25078653e-03, -9.25563844e-03],
       [-1.71756206e-05, -2.67587055e-03, -8.84531134e-03]]), 'b1': array([[0.02373217],
       [0.51851609],
       [0.34590606]]), 'W2': array([[0.88314705, 0.20716934, 0.49025206]]), 'b2': array([[-0.03956218]])}`
       
`sequence= [array([[-2.25577063e-03,  4.06706987e-03,  1.06554360e-02],
       [ 1.33614910e-02, -8.25078653e-03, -9.25563844e-03],
       [-1.71756206e-05, -2.67587055e-03, -8.84531134e-03]]), array([[0.88314705, 0.20716934, 0.49025206]]), array([[0.02373217],
       [0.51851609],
       [0.34590606]]), array([[-0.03956218]])]`

In [ ]:
def cv_to_body(adn):
    """
    cv ADN aka sequence to body aka parameters_set
    """
    params = {}
    params["W1"] = np.reshape(adn[:9], (3, 3))
    params["W2"] = np.reshape(adn[9:12], (1, 3))
    params["b1"] = np.reshape(adn[12:15], (3, 1))
    params["b2"] = np.reshape(adn[15], (1, 1))
    return params

`adn= [-0.00110909, -0.01222481639373562, 0.00053467, 0.00384994, 0.01073706, -0.01976522, 0.01165224282105234, -0.00588098, -0.02677469, 0.06930708, -0.6338238, 0.8109224, 0.39007805, 0.34575972, 0.92335005, 0.35678164]`

`params= {'W1': array([[-0.00110909, -0.01222482,  0.00053467],
       [ 0.00384994,  0.01073706, -0.01976522],
       [ 0.01165224, -0.00588098, -0.02677469]]), 'W2': array([[ 0.06930708, -0.6338238 ,  0.8109224 ]]), 'b1': array([[0.39007805],
       [0.34575972],
       [0.92335005]]), 'b2': array([[0.35678164]])}`


## 1. Initial population


Ta cần 1 hàm tạo ra generation đầu tiên, đây là các cá thể được tạo random. Thực tế khi chạy rồi lúc có thể có những lần fail hoàn toàn do generation đầu quá tệ.

In [5]:
from dino_nn import initialize_parameters


def genesis(pop_size=POP_SIZE):
    trex_clan = [initialize_parameters(
        N_X, N_H, N_Y) for i in range(pop_size)]
    trex_clan = np.array(trex_clan)
    return trex_clan

display(genesis())

array([{'W1': array([[-0.00302326, -0.00556239,  0.00522661],
              [-0.00376682,  0.01254634, -0.01656048],
              [ 0.00080338,  0.00949067, -0.00169025]]), 'b1': array([[0.67816698],
              [0.52777615],
              [0.5239842 ]]), 'W2': array([[ 1.39639392, -0.65520626,  0.02656992]]), 'b2': array([[0.00858657]])},
       {'W1': array([[-0.00736442, -0.00571794,  0.00260728],
              [ 0.00894329, -0.00228684,  0.00942701],
              [-0.01749506,  0.00028303,  0.00400906]]), 'b1': array([[-0.39491711],
              [ 0.23608448],
              [ 0.68964694]]), 'W2': array([[ 1.62645282, -0.0363814 , -0.18950125]]), 'b2': array([[-0.04581984]])},
       {'W1': array([[-0.01570869, -0.01025981,  0.00520108],
              [ 0.0108056 ,  0.01624868,  0.00561545],
              [-0.00758254, -0.00134151, -0.00725735]]), 'b1': array([[ 1.0143746 ],
              [ 0.83292969],
              [-1.17684671]]), 'W2': array([[ 0.10220025, -1.23213048,  1.1

## 2. Fitness function


Lấy điểm số của từng màn chơi (sử dụng lại function playgame ở dino_nn mỗi lần chơi sẽ trả về điểm số khi kết thúc ván game).


In [ ]:
from dino_nn import play_game

score = play_game


## 3. Selection

##### Ta sẽ tiến hành select_survivals từ điểm fitness của các cá thể, sau đó random_match các cha mẹ lại.

In [ ]:
def select_survivals(tribal, score):
    fitness_scores = np.array(score)
    survival_inds = (-fitness_scores).argsort()[:4] # chuyển sang giá trị âm và sort
    return tribal[survival_inds], survival_inds


def random_match(random_set=RANDOM_SET):
    number = -1
    curr_number = sum(random_set)
    lucky_number = np.random.randint(0, curr_number)
    while lucky_number < curr_number:
        number += 1
        curr_number -= random_set[number]
    return number


tribal= [{'W1': array([[ 0.00186908,  0.00170099, -0.00885588],
        [-0.02077583, -0.00893555,  0.0129884 ],
        [ 0.01191064,  0.02585975,  0.00346434]]), 'b1': array([[0.51425576],
        [0.59653082],
        [0.3704624 ]]), 'W2': array([[0.36238972, 1.63961405, 1.19425348]]), 'b2': array([[0.04960243]])}
 {'W1': array([[ 0.00451642, -0.01137283,  0.00313144],
        [-0.00628801, -0.01474175,  0.01487595],
        [-0.00146567,  0.00586895, -0.00914562]]), 'b1': array([[0.05086135],
        [0.40475606],
        [0.20840084]]), 'W2': array([[ 0.07150403, -3.7484707 , -0.15297691]]), 'b2': array([[-0.11427518]])}
 {'W1': array([[-0.00011143,  0.00400506, -0.00808964],
        [ 0.00577842, -0.00897957, -0.00338029],
        [-0.00950392, -0.0013769 , -0.0021641 ]]), 'b1': array([[0.47845439],
        [0.56633776],
        [0.28383487]]), 'W2': array([[-0.20242944, -0.33552417,  1.58525568]]), 'b2': array([[0.13771654]])}
 {'W1': array([[-0.00140887,  0.0052579 ,  0.01415507],
        [-0.00325708, -0.01272355,  0.00185065],
        [-0.00306932,  0.00741625, -0.00568358]]), 'b1': array([[ 0.06478614],
        [ 0.18136403],
        [-0.24832133]]), 'W2': array([[ 0.1080681 ,  0.71073654, -1.42879073]]), 'b2': array([[0.08301525]])}
 {'W1': array([[-0.0096537 ,  0.00512624,  0.01394242],
        [-0.00592077, -0.00242889, -0.01289153],
        [ 0.00240955, -0.00792998, -0.00760194]]), 'b1': array([[ 0.1745851 ],
        [-0.21110215],
        [ 0.73588262]]), 'W2': array([[-0.60193542,  1.0245507 ,  1.03024122]]), 'b2': array([[-0.11328447]])}
 {'W1': array([[ 0.00024512,  0.0020366 , -0.00153019],
        [ 0.00501012,  0.00170439,  0.00421495],
        [-0.00428756, -0.0086934 ,  0.01128537]]), 'b1': array([[ 0.65959463],
        [ 0.15417807],
        [-0.92662122]]), 'W2': array([[ 0.21744948, -0.38402893,  1.72976247]]), 'b2': array([[0.03531734]])}
 {'W1': array([[-0.00844737, -0.0055683 ,  0.00054114],
        [ 0.00118754,  0.01109476,  0.00394489],
        [ 0.00052328,  0.00534172,  0.00865382]]), 'b1': array([[ 0.64607051],
        [-0.27456014],
        [ 0.29132737]]), 'W2': array([[-0.641904  , -3.18930404, -0.54487562]]), 'b2': array([[-0.09343674]])}
 {'W1': array([[-0.00675471,  0.00441748,  0.02169352],
        [-0.00256629,  0.00180769,  0.01340068],
        [-0.00795965, -0.01323201, -0.0006808 ]]), 'b1': array([[ 0.19210553],
        [ 0.92382511],
        [-0.35544641]]), 'W2': array([[0.0864048 , 0.86366838, 0.06522612]]), 'b2': array([[0.20571457]])}
 {'W1': array([[-0.00168429, -0.00846168,  0.00332377],
        [ 0.00164353, -0.00209885, -0.01515935],
        [-0.00238887,  0.00032799,  0.00191436]]), 'b1': array([[-0.05438533],
        [-0.2999632 ],
        [-0.11946396]]), 'W2': array([[-2.13250307, -0.01159749, -0.75998168]]), 'b2': array([[-0.31209534]])}
 {'W1': array([[ 0.00725192,  0.01736422, -0.00573289],
        [ 0.00046444, -0.01527393, -0.00717709],
        [ 0.01550293,  0.00736765, -0.00717323]]), 'b1': array([[ 0.66015564],
        [ 0.11603796],
        [-0.26644995]]), 'W2': array([[-0.529538  ,  1.25577917,  1.44697987]]), 'b2': array([[-0.05917305]])}
 {'W1': array([[-0.01001122, -0.00652026,  0.00380874],
        [-0.00573221, -0.02189533,  0.00211367],
        [ 0.01696092,  0.00966658, -0.00089865]]), 'b1': array([[-0.85489078],
        [ 1.07203556],
        [-0.14380816]]), 'W2': array([[ 1.31447512,  0.2233501 , -0.90156512]]), 'b2': array([[-0.05230664]])}
 {'W1': array([[-0.01180791,  0.01189518,  0.00996632],
        [ 0.0120103 , -0.00965826,  0.00416507],
        [ 0.00736227, -0.00714388,  0.00368161]]), 'b1': array([[ 0.45626667],
        [-0.17731061],
        [-0.25890109]]), 'W2': array([[-0.54372683,  1.1705041 ,  1.26247338]]), 'b2': array([[0.08388264]])}]
score= [49, 43, 1772, 43, 43, 153, 43, 43, 51, 43, 43, 43]
survivals= [{'W1': array([[-0.00011143,  0.00400506, -0.00808964],
        [ 0.00577842, -0.00897957, -0.00338029],
        [-0.00950392, -0.0013769 , -0.0021641 ]]), 'b1': array([[0.47845439],
        [0.56633776],
        [0.28383487]]), 'W2': array([[-0.20242944, -0.33552417,  1.58525568]]), 'b2': array([[0.13771654]])}
 {'W1': array([[ 0.00024512,  0.0020366 , -0.00153019],
        [ 0.00501012,  0.00170439,  0.00421495],
        [-0.00428756, -0.0086934 ,  0.01128537]]), 'b1': array([[ 0.65959463],
        [ 0.15417807],
        [-0.92662122]]), 'W2': array([[ 0.21744948, -0.38402893,  1.72976247]]), 'b2': array([[0.03531734]])}
 {'W1': array([[-0.00168429, -0.00846168,  0.00332377],
        [ 0.00164353, -0.00209885, -0.01515935],
        [-0.00238887,  0.00032799,  0.00191436]]), 'b1': array([[-0.05438533],
        [-0.2999632 ],
        [-0.11946396]]), 'W2': array([[-2.13250307, -0.01159749, -0.75998168]]), 'b2': array([[-0.31209534]])}
 {'W1': array([[ 0.00186908,  0.00170099, -0.00885588],
        [-0.02077583, -0.00893555,  0.0129884 ],
        [ 0.01191064,  0.02585975,  0.00346434]]), 'b1': array([[0.51425576],
        [0.59653082],
        [0.3704624 ]]), 'W2': array([[0.36238972, 1.63961405, 1.19425348]]), 'b2': array([[0.04960243]])}]
survival_inds= [2 5 8 0]

## 4. Crossover

##### Ta tạo ra hàm crossver để tạo ra 1 cá thể con từ 1 cặp cha mẹ cho trước.

In [ ]:
def crossver(adam, eva):
    weight_adam = adam[:12]
    weight_eva = eva[:12]
    bias_adam = adam[12:]
    bias_eva = eva[12:]

    cut_1 = np.random.randint(0, 12)
    cut_2 = np.random.randint(12, 16) - 12

    childs = []
    childs.append(weight_adam[:cut_1] + weight_eva[cut_1:] + bias_adam[:cut_2] + bias_eva[cut_2:])
    childs.append(weight_adam[:cut_1] + weight_eva[cut_1:] + bias_eva[:cut_2] + bias_adam[cut_2:])
    childs.append(weight_eva[:cut_1] + weight_adam[cut_1:] + bias_adam[:cut_2] + bias_eva[cut_2:])
    childs.append(weight_eva[:cut_1] + weight_adam[cut_1:] + bias_eva[:cut_2] + bias_adam[cut_2:])
    selected_child = np.random.randint(0, 4)
    return childs[selected_child]

# 5. Mutation

##### Ta cũng quy định hàm do_mutation để thực hiện tạo đột biến cho các cá thể. Cách tạo đột biến mình có mô tả phía trên.

In [ ]:
def do_mutation(child, mutation_prob = MUTATION_PROB):
    mutation_rate = np.random.random(16)
    new_child = child[:]
    for ind in range(16):
        if mutation_rate[ind] < mutation_prob:
            if ind < 9:
                new_child[ind] += np.random.randn() * MUTATION_RANGE[0]
            elif ind < 12:
                new_child[ind] += np.random.randn() * MUTATION_RANGE[1]
            elif ind < 15:
                new_child[ind] += np.random.randn() * MUTATION_RANGE[2]
            else:
                new_child[ind] += np.random.randn() * MUTATION_RANGE[3]
    return new_child

Tiếp đó ta sẽ liên tục breed_a_child cho tới khi đạt tới size yêu cầu của generation (theo hàm gen_to_max_size)

In [ ]:
def breed_a_child(survivals):
    """
    chon 2 cha me tu survivals va crossver
    sau do cho child mutation
    """
    adam_ind = random_match(RANDOM_SET)
    eva_ind = random_match(RANDOM_SET)
    expected_cain = crossver(survivals[adam_ind], survivals[eva_ind])
    real_cain = do_mutation(expected_cain)
    return real_cain


def gen_to_max_size(survivals, pop_size=POP_SIZE):
    curr_len = N_SIZE
    dna_survivals = map(cv_to_sequence, survivals)
    dna_tribal = dna_survivals[:]
    while curr_len < pop_size:
        new_born = breed_a_child(dna_survivals)
        dna_tribal.append(new_born)
        curr_len += 1
    new_gen = map(cv_to_body, dna_tribal)
    new_gen = np.array(new_gen)
    return new_gen

![Alt text](image_mutation_cylcle.png)

# Evolve

##### Cuối cùng là hàm evolve để tập hợp các hàm phía trên, tiến hành train liên tục và chuyển sang các generation khác. Các thông tin khi chạy đều được lưu vào log.

In [ ]:
def evolve():
    adam_eva = genesis(POP_SIZE)
    curr_gen = adam_eva.copy()
    count_gen = 0
    while True:
        log1.info("-------------------------------------------------------")
        log2.info("-------------------------------------------------------")
        print("-------------------------------------------------------")
        log1.info("Generation: {}".format(count_gen))
        log1.info("a new day has come")
        score = []
        for trex in curr_gen:
            count_cactus = capturing_objects.play_game(trex)
            score.append(count_cactus)
        log1.info(score)
        survivals, survival_inds = select_survivals(curr_gen, score)
        log1.info(survival_inds)
        log1.info("genarating next gen")
        curr_gen = gen_to_max_size(survivals, POP_SIZE)
        count_gen += 1
        log2.info(survivals)
        if max(score) > 15:
            log1.info(survivals)
            continue
        elif count_gen % 10 == 0:
            log1.info(survivals)
        log1.info("")